# Kerasでのネットワーク実装

手作りのニューラルネットワーク専用のフレームワーク「Keras」で構築しよう。まず、必要なパッケージを追加しよう

In [1]:
import numpy as np

# 入力、結合、活性化の「レイヤ」
from keras.layers import Input, Dense, Activation

# モデル
from keras.models import Model

# 活性化関数
from keras.activations import sigmoid

## モデル構築

Kerasのモデルは「レイヤ」で構築する。ネットワークの各パーツにたいして、適するレイヤがある：

- Input: 入力レイヤ。入力の数を定義する
- Dense: 全結合レイヤ。前レイヤの出力と本レイヤの出力をすべてつながるり、荷重を与える
- Activation：活性レイヤ。前レイヤの出力の出力に活性化関数で処理する

In [2]:
def create_model():
    
    # 入力レイヤ
    x = Input(shape=(2), dtype=float)
    
    # 全結合レイヤ（荷重）出力が１つ
    w = Dense(units=1)(x)
    
    # 活性化レイヤ。活性化関数はsigmoid
    y = Activation(sigmoid)(w)
    
    # モデル
    model = Model(inputs=x, outputs=y)
    
    return model

In [3]:
# 確認しましょう
model = create_model()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 3         
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


上記の出力を解析してみると…

`input_1 (InputLayer)        [(None, 2)]               0 `

入力レイヤは入力の数は２つがある。`None`はデータの数を示している。ANDとORの課題で、データの数は４点があるが、現時点でまだ確定ではないので`None`として表示されている。入力レイヤは学習できるパラメータはないので、パラメータの数は0である。

`dense (Dense)               (None, 1)                 3`

全結合レイヤの出力は１つしかない。上記と同様、まだ確定されていないデータの数は`None`で表示。なお、パラメータの数は3となる（荷重２つとバイアス１つ）

`activation (Activation)     (None, 1)                 0`

最後に活性化関数があり、出力は１つです。前の出力を処理するだけで、学習パラメータがない。

In [4]:
# 中身を確認
print (model.get_weights())

[array([[0.13836646],
       [1.3095194 ]], dtype=float32), array([0.], dtype=float32)]


## 推論
このまま（学習なし）で推論をしてみると：

In [5]:
x = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
], dtype=float)

y_pred = model.predict(x)
print(y_pred)

[[0.5       ]
 [0.7874327 ]
 [0.53453654]
 [0.80967283]]


## 学習

推論はまったく当たらないので、学習させましょう！

まず、kerasでは、そのまま学習できない。その先、`create_model()`で作ったモデルを「コンパイル」しなければならない。コンパイルすることにて、モデルをGPUで実行できるようになる

In [6]:
model.compile(optimizer="sgd", loss="mean_squared_error")

上記の命令を解説すると：

- `optimizer`:「最適化方法」という意味であり、勾配関数のことを表している。`sgd`と`Adam`をよく使われている
- `loss`:損失関数のこと。課題により、適する損失関数がありが、とりあえず引き続きに「平均二乗誤差」を利用

なお、学習は、以下の命令で実装：

`model.fit(x, y_true, batch_size=1, epochs=3000)`

- `x`:入力データ
- `y_true`:正しい答えとして期待しているラベル
- `epochs`:学習する回数
- `batch_size`:「バッチサイズ」は、入力データを何点を一緒にまとめて学習させるのか

In [7]:
# 学習データを用意
y_true = np.array([
    [0], 
    [0], 
    [0], 
    [1]
], dtype = float)


model.fit(x, y_true, batch_size=1, epochs=3000)

Epoch 1/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.2977
Epoch 2/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.2962
Epoch 3/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.2947
Epoch 4/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.2932
Epoch 5/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.2917
Epoch 6/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.2903
Epoch 7/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.2888
Epoch 8/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.2874
Epoch 9/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.2859
Epoch 10/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.2845
Epoch 11/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.2831
Epoch 12/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.2817
Epoch 13/3000
4/4 [==============================

4/4 [==============================] - 0s 1ms/step - loss: 0.1959
Epoch 104/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1954
Epoch 105/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1948
Epoch 106/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1943
Epoch 107/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1938
Epoch 108/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1933
Epoch 109/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1927
Epoch 110/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1922
Epoch 111/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1917
Epoch 112/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1912
Epoch 113/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1907
Epoch 114/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1902
Epoch 115/3000
4/4 [=======================

4/4 [==============================] - 0s 2ms/step - loss: 0.1614
Epoch 204/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1612
Epoch 205/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1610
Epoch 206/3000
4/4 [==============================] - 0s 997us/step - loss: 0.1608
Epoch 207/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1605
Epoch 208/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1603
Epoch 209/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1601
Epoch 210/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1599
Epoch 211/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1597
Epoch 212/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1595
Epoch 213/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1593
Epoch 214/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1591
Epoch 215/3000
4/4 [=====================

Epoch 304/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1450
Epoch 305/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1448
Epoch 306/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1447
Epoch 307/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1446
Epoch 308/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1445
Epoch 309/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1443
Epoch 310/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1442
Epoch 311/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1441
Epoch 312/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1440
Epoch 313/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1438
Epoch 314/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1437
Epoch 315/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1436
Epoch 316/3000
4/4 [========

4/4 [==============================] - 0s 1ms/step - loss: 0.1337
Epoch 405/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1336
Epoch 406/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1335
Epoch 407/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1334
Epoch 408/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1333
Epoch 409/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1333
Epoch 410/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1332
Epoch 411/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1331
Epoch 412/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1330
Epoch 413/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1329
Epoch 414/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1328
Epoch 415/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1327
Epoch 416/3000
4/4 [=======================

...

4/4 [==============================] - 0s 2ms/step - loss: 0.0532
Epoch 2592/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0532
Epoch 2593/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0532
Epoch 2594/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0532
Epoch 2595/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0532
Epoch 2596/3000
4/4 [==============================] - 0s 970us/step - loss: 0.0532
Epoch 2597/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0532
Epoch 2598/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0531
Epoch 2599/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0531
Epoch 2600/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0531
Epoch 2601/3000
4/4 [==============================] - 0s 824us/step - loss: 0.0531
Epoch 2602/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0531
Epoch 2603/3000
4/4 [=======

4/4 [==============================] - 0s 1ms/step - loss: 0.0518
Epoch 2691/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0518
Epoch 2692/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0518
Epoch 2693/3000
4/4 [==============================] - 0s 996us/step - loss: 0.0518
Epoch 2694/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0517
Epoch 2695/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0517
Epoch 2696/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0517
Epoch 2697/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0517
Epoch 2698/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0517
Epoch 2699/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0517
Epoch 2700/3000
4/4 [==============================] - 0s 810us/step - loss: 0.0517
Epoch 2701/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0516
Epoch 2702/3000
4/4 [=======

4/4 [==============================] - 0s 2ms/step - loss: 0.0504
Epoch 2790/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0504
Epoch 2791/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0504
Epoch 2792/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0504
Epoch 2793/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0504
Epoch 2794/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0503
Epoch 2795/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0503
Epoch 2796/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0503
Epoch 2797/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0503
Epoch 2798/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0503
Epoch 2799/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0503
Epoch 2800/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0503
Epoch 2801/3000
4/4 [===========

4/4 [==============================] - 0s 996us/step - loss: 0.0491
Epoch 2890/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0491
Epoch 2891/3000
4/4 [==============================] - 0s 0s/step - loss: 0.0491
Epoch 2892/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.0490
Epoch 2893/3000
4/4 [==============================] - 0s 239us/step - loss: 0.0490
Epoch 2894/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0490
Epoch 2895/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0490
Epoch 2896/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0490
Epoch 2897/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0490
Epoch 2898/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0490
Epoch 2899/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0490
Epoch 2900/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0489
Epoch 2901/3000
4/4 [========

4/4 [==============================] - 0s 0s/step - loss: 0.0478
Epoch 2989/3000
4/4 [==============================] - 0s 4ms/step - loss: 0.0478
Epoch 2990/3000
4/4 [==============================] - 0s 996us/step - loss: 0.0478
Epoch 2991/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0478
Epoch 2992/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0478
Epoch 2993/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0478
Epoch 2994/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0478
Epoch 2995/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0477
Epoch 2996/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0477
Epoch 2997/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0477
Epoch 2998/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.0477
Epoch 2999/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.0477
Epoch 3000/3000
4/4 [==========

In [8]:
#結果を確認
y_pred = model.predict(x)
print(y_pred)

[[0.03512022]
 [0.2338548 ]
 [0.22938362]
 [0.71397316]]


## 改善方法：適する損失関数を選ぶ

「平均二乗誤差」普段に「距離」が表している（正解数値からどれぐらい離れるのか？）。実数の場合はこれで良いと思っていいのが、学習データはバイナリ（0か1）の場合、kerasがもと良い損失関数を提供する。それは`binary_crossentropy`という。

それでは、関数を変えて、確かめてみよう～

In [9]:
# もう一度乱数を固定、モデルを構築する
model = create_model()
model.compile(optimizer="sgd", loss="binary_crossentropy")

In [10]:
# 学習実行
model.fit(x, y_true, batch_size=1, epochs=3000)

Epoch 1/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.7815
Epoch 2/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.7742
Epoch 3/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.7671
Epoch 4/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.7602
Epoch 5/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.7534
Epoch 6/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.7468
Epoch 7/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.7403
Epoch 8/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.7340
Epoch 9/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.7279
Epoch 10/3000
4/4 [==============================] - 0s 997us/step - loss: 0.7218
Epoch 11/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.7159
Epoch 12/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.7102
Epoch 13/3000
4/4 [============================

4/4 [==============================] - 0s 997us/step - loss: 0.4956
Epoch 104/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.4947
Epoch 105/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.4938
Epoch 106/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.4929
Epoch 107/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.4921
Epoch 108/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.4912
Epoch 109/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.4904
Epoch 110/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.4896
Epoch 111/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.4887
Epoch 112/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.4879
Epoch 113/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.4871
Epoch 114/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.4863
Epoch 115/3000
4/4 [=====================

4/4 [==============================] - 0s 996us/step - loss: 0.4335
Epoch 205/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.4331
Epoch 206/3000
4/4 [==============================] - 0s 997us/step - loss: 0.4326
Epoch 207/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.4321
Epoch 208/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.4316
Epoch 209/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.4312
Epoch 210/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.4307
Epoch 211/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.4303
Epoch 212/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.4298
Epoch 213/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.4293
Epoch 214/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.4289
Epoch 215/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.4284
Epoch 216/3000
4/4 [===================

4/4 [==============================] - 0s 1ms/step - loss: 0.3922
Epoch 305/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.3918
Epoch 306/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.3914
Epoch 307/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.3911
Epoch 308/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.3907
Epoch 309/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.3903
Epoch 310/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.3900
Epoch 311/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.3896
Epoch 312/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.3893
Epoch 313/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.3889
Epoch 314/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.3885
Epoch 315/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.3882
Epoch 316/3000
4/4 [=======================

4/4 [==============================] - 0s 2ms/step - loss: 0.3588
Epoch 406/3000
4/4 [==============================] - 0s 997us/step - loss: 0.3585
Epoch 407/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.3582
Epoch 408/3000
4/4 [==============================] - 0s 997us/step - loss: 0.3579
Epoch 409/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.3576
Epoch 410/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.3573
Epoch 411/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.3570
Epoch 412/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.3567
Epoch 413/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.3564
Epoch 414/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.3561
Epoch 415/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.3558
Epoch 416/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.3555
Epoch 417/3000
4/4 [===================

...

Epoch 2601/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1347
Epoch 2602/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1346
Epoch 2603/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1346
Epoch 2604/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1346
Epoch 2605/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1345
Epoch 2606/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1345
Epoch 2607/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1344
Epoch 2608/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1344
Epoch 2609/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1344
Epoch 2610/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1343
Epoch 2611/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1343
Epoch 2612/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1343
Epoch 2613/3000


4/4 [==============================] - 0s 2ms/step - loss: 0.1310
Epoch 2701/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1310
Epoch 2702/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1310
Epoch 2703/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1309
Epoch 2704/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1309
Epoch 2705/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1308
Epoch 2706/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1308
Epoch 2707/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1308
Epoch 2708/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1307
Epoch 2709/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1307
Epoch 2710/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1307
Epoch 2711/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1306
Epoch 2712/3000
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 0.1275
Epoch 2801/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1275
Epoch 2802/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1275
Epoch 2803/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1274
Epoch 2804/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1274
Epoch 2805/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1274
Epoch 2806/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1273
Epoch 2807/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1273
Epoch 2808/3000
4/4 [==============================] - ETA: 0s - loss: 0.212 - 0s 2ms/step - loss: 0.1273
Epoch 2809/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1272
Epoch 2810/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1272
Epoch 2811/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1272
Epoch 28

4/4 [==============================] - 0s 1ms/step - loss: 0.1243
Epoch 2900/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1242
Epoch 2901/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1242
Epoch 2902/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.1242
Epoch 2903/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1241
Epoch 2904/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1241
Epoch 2905/3000
4/4 [==============================] - 0s 3ms/step - loss: 0.1241
Epoch 2906/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1240
Epoch 2907/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1240
Epoch 2908/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1240
Epoch 2909/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1239
Epoch 2910/3000
4/4 [==============================] - 0s 2ms/step - loss: 0.1239
Epoch 2911/3000
4/4 [===========

4/4 [==============================] - 0s 2ms/step - loss: 0.1211
Epoch 3000/3000
4/4 [==============================] - 0s 1ms/step - loss: 0.1211


In [11]:
#結果を確認
y_pred = model.predict(x)
print(y_pred)

[[0.00489625]
 [0.12871137]
 [0.12949377]
 [0.81705654]]


In [12]:
print(model.get_weights())

[array([[3.4089322],
       [3.4019735]], dtype=float32), array([-5.3143744], dtype=float32)]


## 練習：ORを学習させる

## 練習：XORを学習させる